In [1]:
import h5py as h5
from h5glance import H5Glance
import numpy as np
import gvar as gv
import pandas as pd
import os 
import sys
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib notebook
#import h5ls
import lsqfit
import matplotlib
from matplotlib.backends.backend_pdf import PdfPages
import yaml
from pathlib import Path

In [2]:
#import local files
import data_loader as dl
import fitter as fit
import fit_manager as fm

matplotlib.rcParams['figure.figsize'] = [10, 8]

In [3]:
os.getcwd()

'/Users/grantdb/lqcd/nucleon_sigma'

In [9]:
#ens = 
p_dict = {
    'abbr' : 'a06m310L', #CHANGE THIS
    'part' : 'nucleon', #CHANGE THIS # 'proton'
    'particles' : ['nucleon'],#'axial_fh_num', 'vector_fh_num'],
    'sinks' : ['dir', 'smr'],

    't_range' : {
        'corr' : [7, 12], #change these
        'gA'   : [4, 12],
        'gV'   : [4, 12],
    },
    'n_states' : {
        'corr' : 2,
        'gA'   : 3,
        'gV'   : 3, #for a15 ensemble, 4 gives less fit curvature 
    },
    
    'make_plots' : True,
    'save_prior' : False,
    
    'show_all' : True,
    'show_many_states' : False, # This doesn't quite work as expected: keep false
    'use_prior' : True
}

In [10]:
file = './data/c51_2pt_octet_decuplet.h5'
with h5.File(file,"r") as f:
    ensembles = {}
    ensembles = list(f.keys())
    print(ensembles,len(ensembles))

['a06m220L', 'a06m310', 'a06m310L', 'a09m135', 'a09m220', 'a09m220_new_old', 'a09m220_o', 'a09m260', 'a09m310', 'a09m310_new_old', 'a09m310_o', 'a09m350', 'a09m400', 'a12m130', 'a12m180L', 'a12m180S', 'a12m220', 'a12m220L', 'a12m220S', 'a12m220XL', 'a12m220_ms', 'a12m220_new_old', 'a12m220_o', 'a12m260', 'a12m310', 'a12m310L', 'a12m310XL', 'a12m350', 'a12m400', 'a15m130', 'a15m135XL', 'a15m220', 'a15m260', 'a15m310', 'a15m310L', 'a15m310L_new_old', 'a15m310L_o', 'a15m350', 'a15m400'] 39


In [11]:


def get_corr(file_h5,abbr):
    data = {}
    with h5.File(file_h5,"r") as f:
        path = "/"+ abbr 
        particles = f[path].keys()
        for part in particles:
            cfgs = np.array(f[path+"/" + part][()])
            print(cfgs)
            data[part] = cfgs.real # .real for the _hp ensembles 
    return data

In [12]:
o_data = get_corr(file,p_dict['abbr'])
#o_data

[[[[2.03933337e-09+2.8027247e-11j]
   [5.88697580e-09+1.2875920e-10j]]

  [[7.52585265e-07-8.2452101e-11j]
   [4.42992132e-06-2.1188522e-09j]]

  [[2.45339521e-07+3.0797240e-11j]
   [1.15791136e-06-8.2648055e-10j]]

  ...

  [[3.17082467e-08-8.5228907e-12j]
   [1.71300471e-07+3.9713430e-10j]]

  [[1.02847061e-07-1.7892222e-10j]
   [6.47880768e-07+4.5236592e-10j]]

  [[4.09750697e-07-8.7169222e-10j]
   [3.00994679e-06-2.7928770e-10j]]]


 [[[2.62291189e-09-1.6062235e-10j]
   [7.59881846e-09-3.3130204e-10j]]

  [[7.55762812e-07+6.2664568e-11j]
   [4.46451304e-06-1.0905072e-09j]]

  [[2.43990968e-07+2.8631922e-10j]
   [1.16220974e-06-7.3904116e-10j]]

  ...

  [[3.04051611e-08+6.5582267e-10j]
   [1.68615458e-07+1.1379540e-09j]]

  [[9.97322829e-08+7.4482648e-10j]
   [6.40934786e-07+8.6574453e-10j]]

  [[4.04277898e-07+4.6168522e-10j]
   [2.99962903e-06+4.4673765e-11j]]]


 [[[2.24985897e-09-5.2814912e-11j]
   [6.53070886e-09-1.1678773e-10j]]

  [[7.35589424e-07-1.2095647e-09j]
   [4.37227

In [13]:
#H5Glance(file)

In [14]:
#load data into dictionaries to prepare for gvar averaging

nucleon_corr_data = None
if 'proton' in p_dict['particles']:
    nucleon_corr_data = {}
    for i,s in enumerate(p_dict['sinks']):
        nucleon_corr_data[s] = o_data[p_dict['part']][:,:,i,0] #change
        
axial_fh_num_data = None
if 'axial_fh_num' in p_dict['particles']:
    axial_fh_num_data = {}
    for i,s in enumerate(p_dict['sinks']):
        axial_fh_num_data[s] = o_data[p_dict['part']+'_A3'][:,:,i,0] #change

vector_fh_num_data = None
if 'vector_fh_num' in p_dict['particles']:
    vector_fh_num_data = {}
    for i,s in enumerate(p_dict['sinks']):
        vector_fh_num_data[s] = o_data[p_dict['part']+'_V4'][:,:,i,0] #change        

## pull in priors from csv in ens dirs, change csv as see fit ##

In [15]:
prior_nucl = {}

path = os.path.normpath("./priors/{0}/{1}/prior_nucl.csv".format(p_dict['abbr'],p_dict['part']))
df = pd.read_csv(path, index_col=0).to_dict()
for key in list(df.keys()):
    if key == 'g_A_nm' or key == 'g_V_nm':
        fh_ratio = list(df[key].values())
        size = int(np.sqrt(len(list(df[key].values()))))
        fh_ratio = np.reshape(fh_ratio, (size, size))
        prior_nucl[key] = fh_ratio
    else:
        length = int(np.sqrt(len(list(df[key].values()))))
        prior_nucl[key] = list(df[key].values())[:length]

prior_nucl = gv.gvar(prior_nucl)




FileNotFoundError: [Errno 2] No such file or directory: 'priors/a06m310L/nucleon/prior_nucl.csv'

In [11]:
# prior_gA = {}

# path = os.path.normpath("./nucleon-axial-coupling-db/{0}/{1}/prior_gA.csv".format(p_dict['abbr'],p_dict['part']))
# df = pd.read_csv(path, index_col=0).to_dict()
# for key in list(df.keys()):
#     if key == 'g_A_nm' or key == 'g_V_nm':
#         fh_ratio = list(df[key].values())
#         size = int(np.sqrt(len(list(df[key].values()))))
#         fh_ratio = np.reshape(fh_ratio, (size, size))
#         prior_gA[key] = fh_ratio
#     else:
#         length = int(np.sqrt(len(list(df[key].values()))))
#         prior_gA[key] = list(df[key].values())[:length]

# prior_gA = gv.gvar(prior_gA)

In [12]:
# prior_gV = {}

# path = os.path.normpath("./nucleon-axial-coupling-db/{0}/{1}/prior_gV.csv".format(p_dict['abbr'],p_dict['part']))
# df = pd.read_csv(path, index_col=0).to_dict()
# for key in list(df.keys()):
#     if key == 'g_A_nm' or key == 'g_V_nm':
#         fh_ratio = list(df[key].values())
#         size = int(np.sqrt(len(list(df[key].values()))))
#         fh_ratio = np.reshape(fh_ratio, (size, size))
#         prior_gV[key] = fh_ratio
#     else:
#         length = int(np.sqrt(len(list(df[key].values()))))
#         prior_gV[key] = list(df[key].values())[:length]

# prior_gV = gv.gvar(prior_gV)

In [13]:
prior = {**prior_nucl} #USE THIS FOR HP ENSEMBLES 
#prior = {**prior_nucl,**prior_gA,**prior_gV} #USE THIS DICTIONARY FOR ALL OTHER ENSß
prior

{'E': array([0.34(22), 0.58(22), 0.78(22), 0.98(22)], dtype=object),
 'wf_dir': array([0.0(3.3)e-06, 0.0(3.3)e-06, 0.0(3.3)e-06, 0.0(3.3)e-06],
       dtype=object),
 'wf_smr': array([4.4(4.4)e-06, 4.4(4.4)e-06, 4.4(4.4)e-06, 4.4(4.4)e-06],
       dtype=object)}

In [16]:
fit_ensemble = fm.fit_ensemble(t_range=p_dict['t_range'],
                               n_states=p_dict['n_states'], prior=prior, 
                               nucleon_corr_data=nucleon_corr_data, 
                               axial_fh_num_data=axial_fh_num_data,
                               vector_fh_num_data=vector_fh_num_data)
print(fit_ensemble)
if p_dict['save_prior']:
    dl.save_prior(fit_ensemble.make_prior_from_fit(), abbr=p_dict['abbr'])

NameError: name 'fm' is not defined

In [ ]:
fit_ensemble.make_prior_from_fit()

{'wf_dir': array([0.0(4.3)e-07, 0.0(4.3)e-07, 0.0(4.3)e-07, 0.0(4.3)e-07],
       dtype=object),
 'wf_smr': array([8.0(8.0)e-07, 8.0(8.0)e-07, 8.0(8.0)e-07, 8.0(8.0)e-07],
       dtype=object),
 'E': array([0.58(22), 0.89(22), 1.01(22), 1.08(22)], dtype=object)}

In [ ]:
#pickle gvar priors/posteriors from fit using gvar dump
fit_out = fit_ensemble.get_fit()
fit_dump = {}
fit_dump['prior'] = fit_out.prior
fit_dump['p'] = fit_out.p
fit_dump['logGBF'] = fit_out.logGBF
fit_dump['Q'] = fit_out.Q
print(fit_dump)

gv.dump(fit_dump,'ga_fit_results/{1}/{0}_{1}.p'.format(p_dict['part'],p_dict['abbr']))

{'prior': BufferDict([('wf_dir', array([0.0(4.3)e-07, 0.0(4.3)e-07, 0.0(4.3)e-07], dtype=object)), ('wf_smr', array([9.4(9.4)e-07, 9.4(9.4)e-07, 9.4(9.4)e-07], dtype=object)), ('E0', 0.59(22)), ('log(dE)', array([-1.14(69), -2(24)], dtype=object))]), 'p': BufferDict([('wf_dir', array([2.15(29)e-07, 2.77(87)e-07, 1e-22 +- 4.3e-07], dtype=object)), ('wf_smr', array([8.0(1.1)e-07, 1.35(48)e-06, 9.4(9.4)e-07], dtype=object)), ('E0', 0.579(10)), ('log(dE)', array([-1.08(31), -2(24)], dtype=object))]), 'logGBF': 588.2683423570619, 'Q': 0.11841461208626103}


'ga_fit_results/a12m180L/proton_a12m180L.p'

In [ ]:
#reassemble data from above 
#g = gv.load('ga_fit_results/{1}/{0}_{1}.p'.format(p_dict['part'],p_dict['abbr']))
#g

In [ ]:
#comment out plot commands in fit_manager for ga, gv for hp ens
if p_dict['make_plots']:
    plots = fit_ensemble.make_plots()
    output_dir = 'ga_fit_results/{0}/{1}_{0}.pdf'.format(p_dict['abbr'],p_dict['part'])
    output_pdf = PdfPages(output_dir)
    for plot in plots:
        if plot is not None:
            output_pdf.savefig(plot)
    if p_dict['show_many_states']:
        output_pdf.savefig(fit_ensemble.plot_stability(model_type='corr', n_states_array=[1, 2, 3, 4]))
        #output_pdf.savefig(fit_ensemble.plot_stability(model_type='gA', n_states_array=[1, 2, 3, 4]))
        #output_pdf.savefig(fit_ensemble.plot_stability(model_type='gV', n_states_array=[1, 2, 3, 4]))
    output_pdf.close()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## bootstrapping ##

bs resampling and redo fits 

keep track of seed 

gauge average bc approximate integral that way -> physical point

data from configs, resample gauge configs, average them to get result of integral

rand sample m times (gauge field configs) to compute bs sample 

array n x m, average over m (n number of resamples)

each resample n correspond to an integral to be computed by avg over m configurations

https://lsqfit.readthedocs.io/en/latest/lsqfit.html
1) make a large number of “bootstrap copies” of the original input data and prior that differ from each other by random amounts characteristic of the underlying randomness in the original data

2) repeat the entire fit analysis for each bootstrap copy of the data, extracting fit results from each 

3) use the variation of the fit results from bootstrap copy to bootstrap copy to determine an approximate probability distribution (possibly non-gaussian) for the fit parameters and/or functions of them: the results from each bootstrap fit are samples from that distribution.

In [ ]:
def bs_to_gvar(data, bs_list):
    bs_M = data['smr'].shape[0] # bs_M would be the same if you used 'ps' instead
    #bs_list = np.random.randint(low=0, high=bs_M, size=(bs_N, bs_M))
    
    temp_dict = {}
    for key in data.keys():
        temp = data[key][bs_list[0:]
        temp_dict[key] = np.mean(temp, axis=0).real
        
    for k in range(1, bs_N):
        for key in data.keys():
            temp = data[key][bs_list[k:]
            temp_dict[key] = np.vstack((temp_dict[key], np.mean(temp, axis=0).real))
    
    output = {}
    #return temp_dict
    for key in data.keys():
        mean = np.mean(data[key], axis=0)
        unc = np.cov(temp_dict[key], rowvar=False)
        output[key] = gv.gvar(mean,unc)
    return output

nucleon_bs_list = bs_to_gvar(nucleon_corr_data, 500)
axial_bs_list   = bs_to_gvar(axial_fh_num_data,500)
vector_bs_list  = bs_to_gvar(vector_fh_num_data,500)
#nucleon_bs_list


SyntaxError: invalid syntax (<ipython-input-515-3bdba660a7e5>, line 8)

In [ ]:
#b = gv.dataset.bootstrap_iter(nucleon_bs_list)
#print(next(b))b

In [ ]:
bs_list_path = os.path.realpath('./c51_mdwf_hisq_spec.h5')
#H5Glance(bs_list_path)

In [ ]:
with h5.File(bs_list_path, "r") as f: 
    bs = f["/"+ p_dict['abbr']+"/bs_lst"]
    bs_list = np.array(bs)
    print(bs_list)

OSError: Unable to open file (unable to open file: name = '/Users/grantdb/hyperon_ga/c51_mdwf_hisq_spec.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
#bs_list[1,:]

In [ ]:
print(len(bs_list))
# bs_M = nucleon_corr_data['smr'].shape[0] # bs_M would be the same if you used 'ps' instead
bs_N = 1000 # In real-world cases, this would probably be much larger
# bs_list = np.random.randint(low=0, high=bs_M, size=(bs_N, bs_M))
# bs_list
#bs_list[:,:]

NameError: name 'bs_list' is not defined

In [ ]:
n_time_slices = nucleon_corr_data['smr'].shape[1] # again, could use 'ps' instead
bs_correlators_gv = []
for k in range(bs_N):
    temp = ({key : nucleon_corr_data[key][bs_list[k,:], :]
                           for key in nucleon_corr_data.keys()})
    
    # convert each bootstrap resample into a correlator with uncertainty
    # by using gv.dataset.avg_data (same function we used to make original data for fit)
    bs_correlators_gv.append(gv.dataset.avg_data(temp))
#bs_correlators_gv

In [ ]:
fit_ensemble_bs = fm.fit_ensemble(t_range=p_dict['t_range'],
                               n_states=p_dict['n_states'], prior=prior, 
                               nucleon_corr_data=bs_correlators_gv, 
                               axial_fh_num_data=axial_fh_num_data,
                               vector_fh_num_data=vector_fh_num_data)
print(fit_ensemble_bs)

In [ ]:
# Make bs_N fits, one for each of the resampled correlators
fit_keys = fit_out.p.keys()
output = {key : [] for key in fit_keys}

for j, bsfit in enumerate(fit_out.bootstrapped_fit_iter(datalist=bs_correlators_gv)):
    for key in fit_keys:
        # Save the best estimate for the central value 
        # of each parameter of each fit
        p = bsfit.pmean[key]
        output[key].append(p)

# # print results -- should be similar to previous results
table = gv.dataset.avg_data(output, bstrap=True)
print(gv.tabulate(table))

In [ ]:
#print(gv.evalcorr(table[key][:2])[1,0])
# measure of how well multi-dimensional Gaussian distributions(regular fit and bootstrapped fit) agree with each other 
# do their means agree within errors for corresponding elements. 
# note they share the same keys, so just loop over those 
# only parts of fit params that overlap are used 
for key in fit_keys:
    chi2 = gv.chi2(table[key],fit_out.p[key])
gv.fmt_chi2(chi2)

In [ ]:
print(fit_ensemble.get_fit())